# Create Heat Map showing use of Taxi Vs FHV

In [1]:
import pandas as pd

import gmaps

# Google developer API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Get Rides Data

In [2]:
rides_df = pd.read_csv("All_TLC_2018Trip.csv")


rides_df.head()


,Pickup_ID,month,count,category
0,1.0,1.0,584.0,Taxi
1,2.0,1.0,4.0,Taxi
2,3.0,1.0,432.0,Taxi
3,4.0,1.0,19656.0,Taxi
4,5.0,1.0,2.0,Taxi


# Import Airport and SportsVenues Data

In [3]:
airport_df = pd.read_csv("airport.csv")

sportsvenues_df  = pd.read_csv("Sportsvenues.csv")

In [4]:
# Show Airports
airport_df

,Airport,Latitude,Longitude
0,John F. Kennedy International Airport,40.641311,-73.778139
1,Newark Liberty International Airport,40.689531,-74.174462
2,LaGuardia Airport,40.776927,-73.873966
3,Westchester County Airport,41.068333,-73.708664
4,Teterboro Airport,40.858332,-74.061497
5,Republic Airport,40.726127,-73.416792
6,Newark international airport,40.690368,-74.177585


In [5]:
# Show Sports Venues
sportsvenues_df  

,Venue,Latitude,Longitude
0,Citi Field,40.7571,-73.8458
1,Yankee Stadium,40.8296,-73.9262
2,Madison Square Garden,40.7505,-73.9934
3,Barclays Center,40.6826,-73.9754
4,MetLife Stadium,40.8128,-74.0742
5,Prudential Center,40.7335,-74.1710


# Get Locations Data

In [6]:
loc_df_orig = pd.read_csv("TLC_Locations.csv")

loc_df = loc_df_orig[["LocationID", "Lat", "Lng"]]


# Mere Data Sets to pull summary by Lat, Lng for google map

In [7]:
rides_df_taxi = rides_df.loc[rides_df['category'] == 'Taxi']

comb_df_taxi = pd.merge(loc_df,rides_df_taxi, left_on ='LocationID', right_on='Pickup_ID', how ='outer' ).fillna(method='ffill')

rides_df_FHV = rides_df.loc[rides_df['category'] == 'FHV']

comb_df_FHV = pd.merge(loc_df,rides_df_FHV, left_on ='LocationID', right_on='Pickup_ID', how ='outer' ).fillna(method='ffill')


In [8]:
comb_df_taxi = comb_df_taxi[["LocationID","Lat","Lng", "month","count", "category"]]

comb_df_FHV = comb_df_FHV[["LocationID","Lat","Lng", "month","count", "category"]]

In [9]:
loc_df =loc_df.drop_duplicates("LocationID")
locations = loc_df[["Lat", "Lng"]].astype(float)

In [10]:

taxi_count = comb_df_taxi.groupby("LocationID")["count"].sum().astype(float)


FHV_count= comb_df_FHV.groupby("LocationID")["count"].sum().astype(float)

len(locations)
len(taxi_count)
#len(FHV_count)
#taxi_count

260

In [11]:
# These variable are not used
total_num = comb_df_taxi["count"].sum()
taxi_weights = taxi_count/total_num
taxi_intensity = taxi_count.max()
taxi_count_df = taxi_count.to_frame()
loc_df_orig.head(5)
taxi_count_df.head(5)

,count
LocationID,
1.0,7960.0
2.0,84.0
3.0,7636.0
4.0,212972.0
5.0,173.0


# Top 10 Ride Areas for Taxi

In [12]:
taxi_show_popular = pd.merge(taxi_count, loc_df_orig, on ="LocationID")
taxi_show_popular = taxi_show_popular[["LocationID", "zone", "borough", "count"]]
taxi_show_popular.sort_values(by=['count'], ascending=False).head(10)

,LocationID,zone,borough,count
236,237.0,Upper East Side South,Manhattan,4257688.0
160,161.0,Midtown Center,Manhattan,3951442.0
235,236.0,Upper East Side North,Manhattan,3883516.0
161,162.0,Midtown East,Manhattan,3609312.0
262,263.0,Yorkville West,Manhattan,3590364.0
185,186.0,Penn Station/Madison Sq West,Manhattan,3491252.0
229,230.0,Times Sq/Theatre District,Manhattan,3485864.0
169,170.0,Murray Hill,Manhattan,3311604.0
47,48.0,Clinton East,Manhattan,3266628.0
233,234.0,Union Sq,Manhattan,3207840.0


# Top 10 - Ride Areas for FHV

In [13]:
FHV_show_popular = pd.merge(FHV_count, loc_df_orig, on ="LocationID")
FHV_show_popular = FHV_show_popular[["LocationID", "zone", "borough", "count"]]
FHV_show_popular.sort_values(by=['count'], ascending=False).head(10)

,LocationID,zone,borough,count
78,79.0,East Village,Manhattan,3805045.0
131,132.0,JFK Airport,Queens,3499619.0
160,161.0,Midtown Center,Manhattan,3439552.0
60,61.0,Crown Heights North,Brooklyn,3344966.0
47,48.0,Clinton East,Manhattan,3304889.0
233,234.0,Union Sq,Manhattan,3193018.0
137,138.0,LaGuardia Airport,Queens,3094901.0
230,231.0,TriBeCa/Civic Center,Manhattan,2812918.0
147,148.0,Lower East Side,Manhattan,2729770.0
163,164.0,Midtown South,Manhattan,2595477.0


# Create Taxi Heat Map

In [27]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer1 = gmaps.heatmap_layer(locations, weights=taxi_count)

#heat_layer = gmaps.heatmap_layer(locations, weights=taxi_count, dissipating=False, max_intensity=taxi_intensity,point_radius = 0.1)

# # Adjust heat_layer setting to help with heatmap dissipating on zoom
# heat_layer.dissipating = False
# heat_layer.max_intensity = 100
# heat_layer.point_radius = 0.01


fig.add_layer(heat_layer1)

# Add Airports

airport_marker_locations = airport_df[["Latitude","Longitude"] ]
airport_markers = gmaps.symbol_layer(airport_marker_locations, fill_color='black', stroke_color='black', scale = 6)
fig.add_layer(airport_markers)

# Add Sports Venues


sports_marker_locations = sportsvenues_df[["Latitude","Longitude"] ]
sports_markers = gmaps.symbol_layer(sports_marker_locations,  fill_color='rgba(200, 0, 0, 0.4)', stroke_color='rgba(200, 0, 0, 0.4)', scale =6)
fig.add_layer(sports_markers)

fig

Figure(layout=FigureLayout(height='420px'))

# Create FHV Heat Map

In [28]:
heat_layer2 = gmaps.heatmap_layer(locations, weights=FHV_count)

fig.add_layer(heat_layer2)

# Add Airports

fig.add_layer(airport_markers)

# Add Sports Venues

fig.add_layer(sports_markers)

fig

Figure(layout=FigureLayout(height='420px'))